# IC/DAQ Lab
In this lab, you will learn to connecto to instruments to perform a measurement of the frequency transmission through an electronic circuit. First, we will learn how to connect to the oscilloscope and read out data. To begin, create the variable `rm`, which is the pyvisa resource manager. Then, connect to the oscilloscope using the ip address and assign the resource to the variable `inst`. Test your connection using the `'*IDN?'` command. Wrap this code into a function that connects to the resource, prints the `*IDN?` query, returns `inst`. Write docstrings for the function. For each function, you should complete the docstring where it says "COMPLETE THIS".

In [ ]:
import pyvisa
import numpy as np
from time import sleep
import matplotlib.pyplot as plt
from simple_fft import simple_fft

In [ ]:
def connect(ip_address = '192.168.1.164'):
    """
    COMPLETE THIS 

    :param ip_address: str, COMPLETE THIS  

    :return inst: pyvisa resource
    """
    # Your code here 
    return inst

inst = connect_oscilloscope()

### Setting up the oscilloscope parameters
Next, we will set some of the oscilloscope settings manually. Fill in the following functions. Use the SDS programming manual to find the appropriate SCPI commands. Write docstrings for each function.

### Note
Several of these parameters can only be set to discrete values. If we try to set a value that is not possible, it will choose the closest value. For example, if we try to set the yscale to 999 mV / div, it will choose 1 V / div. We can ignore this for now, but keep it in mind so you don't run into problems later.

In [ ]:
def set_xscale(inst, xscale):
    """
    Sets the horizontal scale

    :param inst: pyvisa resource
    :param xscale: float, horizontal scale in seconds / div
    """
    inst.write(f'TIME_DIV {xscale}s')
    
def set_xoffset(inst, xoffset):
    """
    Sets the horizontal offset

    :param inst: pyvisa resource
    :param xoffset: float, horizontal scale in seconds
    """
    inst.write(f'HOR_POSITION {xoffset}s')
    
def set_yscale(inst, yscale, ch = 1):
    """
    Sets the vertical scale on the given channel

    :param inst: pyvisa resource
    :param yscale: float, vertical scale in V / div
    :param ch: int, channel index, must be in [1, 2, 3, 4]
    """
    inst.write(f'C{ch}:VOLT_DIV {yscale}V')
    
def set_yoffset(inst, yoffset, ch = 1):
    """
    Sets the vertical offset on the given channel

    :param inst: pyvisa resource
    :param yoffset: float, vertical offset in V
    :param ch: int, channel index, must be in [1, 2, 3, 4]
    """
    inst.write(f'C{ch}:OFFSET {yoffset}v')
    
def set_amplification(inst, amplification, ch = 1):
    """
    Sets the amplification on the given channel

    :param inst: pyvisa resource
    :param amplification: float, amplification
        Must be in [0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100]
    :param ch: int, channel index, must be in [1, 2, 3, 4]
    """
    inst.write(f'C{ch}:ATTENUATION {amplification}')

def set_trigger_mode(inst, mode = 'AUTO'):
    """
    Sets the trigger mode 

    :param inst: pyvisa resource
    :param mode: str, trigger mode
        Must be in ['AUTO', 'NORM', 'SINGLE', 'STOP']
    """
    modes = ['AUTO', 'NORM', 'SINGLE', 'STOP']
    if mode not in modes:
        raise ValueError('Invalid mode')
    inst.write(f'TRIG_MODE {mode}')

### Initialization function
Write a function that initializes the instrument using the following parameters:
<ul>
    <li>xscale: 10 $\mu$s/div</li>
    <li>xoffset: 0 s</li>
    <li>yscale: 1 V/div</li>
    <li>yoffset: 0 V</li>
    <li>amplification: 1X</li>
    <li>trigger mode: AUTO</li>
</ul>
Make sure to set these parameters for all four channels.

In [ ]:
def initialize(inst):
    """ functions
    Initializes the oscilloscope instrument to the following 
    values on every channel 

    xscale        -> 10 us / div
    xoffset       -> 0 s
    yscale        -> 1 V / div 
    yoffset       -> 0 V 
    amplification -> 1X 
    trigger mode  -> AUTO
    """
    set_xscale(inst, 10e-6)
    set_xoffset(inst, 0)
    for ch in range(4):
        set_yscale(inst, 1, ch = ch)
        set_yoffset(inst, 0, ch = ch)
        set_amplification(inst, 1, ch = ch) 
    set_trigger_mode(inst, 'AUTO')
initialize(inst)

### Bonus
For each of the funtions above, write the corresponding 'get' function which returns the value of the parameter.

In [ ]:
def get_xscale(inst):
    """
    Gets the horizontal scale

    :param inst: pyvisa resource
    
    :return xscale: float, horizontal scale in seconds / div
    """
    response = inst.query('TIME_DIV?')
    response = response.split('TDIV ')[1].replace('S\n', '')
    xscale = float(response)
    return xscale
    
def get_xoffset(inst):
    """
    Gets the horizontal offset

    :param inst: pyvisa resource
    
    :return xoffset: float, horizontal scale in seconds
    """
    response = inst.query('HOR_POSITION?')
    response = response.split('HPOS ')[1].replace('S\n', '')
    xoffset = float(response)functions
    return xoffset
    
def get_yscale(inst, ch = 1):
    """
    Gets the vertical scale on the given channel

    :param inst: pyvisa resource
    :param ch: int, channel index, must be in [1, 2, 3, 4]

    :return yscale: float, vertical scale in V / div
    """
    response = inst.query(f'C{ch}:VOLT_DIV?')
    response = response.split('VDIV ')[1].replace('V\n', '')
    yscale = float(response)
    return yscale
    
def get_yoffset(inst, ch = 1):
    """
    Gets the vertical offset on the given channel

    :param inst: pyvisa resource
    :param ch: int, channel index, must be in [1, 2, 3, 4]

    :return yoffset: float, vertical offset in V
    """
    response = inst.query(f'C{ch}:OFFSET?')
    response = response.split('OFST ')[1].replace('V\n', '')
    yoffset = float(response)
    return yoffset
    functions
def get_amplification(inst, ch = 1):
    """
    Gets the amplification on the given channel

    :param inst: pyvisa resource
    :param ch: int, channel index, must be in [1, 2, 3, 4]

    :return amplification: float, amplification
    """
    response = inst.query(f'C{ch}:ATTENUATION?')
    response = response.split('ATTN ')[1].replace('\n', '')
    amplification = float(response)
    return amplification

def get_trigger_mode(inst):
    """
    Gets the trigger mode 

    :param inst: pyvisa resource
    
    :return mode: str, trigger mode
    """
    response = inst.query('TRIG_MODE?')
    response = response.split('TRMD ')[1].replace('\n', '')
    mode = response
    return mode

### Logging
We want to remember the parameters we set, so we need to set up a log file to save these parameters. Write a function that creates the log string, which will later be save to a log file. Include the oscilloscope name at the top of the log file. Write a docstring for the function.

In [ ]:
def create_log(xscale, xoffset, yscale, yoffset, 
               amplification, trigger_mode):
    """
    Creates a log file to keep track of the 
    oscilloscope parameters

    :param xscale: float, horizontal scale in seconds / div
    :param xoffset: float, horizontal scale in seconds
    :param yscale: float, vertical scale in V / div
    :param yoffset: float, vertical offset in V
    :param amplification: float, amplification
    :param trigger_mode: str, trigger mode
    """
    log = "------------- SIGLENT SDS 1104X-E -------------\n" 
    log += f'xscale: {xscale} s / div\n'
    log += f'xoffset: {xoffset} s\n'
    log += f'yscale: {yscale} V / div\n'
    log += f'yoffset: {yoffset} V\n'
    log += f'amplification: {amplification}X\n'
    log += f'trigger mode: {trigger_mode}\n'
    return log 

### Reading data
Next, we will read data from the oscilloscope. First, write a segment of code to get the sample time and sample frequency.

In [ ]:
fsample = inst.query('SARA?')
fsample = float(fsample.split(' ')[1].replace('Sa/s\n', ''))
tsample = 1 / fsample

Now, write a segment of code to get the number of Volts per division and the voltage offset. If you did the bonus earlier, you should already have this code written.

In [ ]:
vdiv = get_yscale(inst)
voffset = get_yoffset(inst)

Now, right a segment of code to receive the data. Instead of using `query` here, we will use `inst.query_binary_values` with the extra argument `datatype 'B'`. This will ensure that the binary values are converted to integers correctly. We will then have to convert the binary values to a voltage using vdiv and voffset. Use the programming manual to make sure you have done this correctly.

In [ ]:
raw_data = inst.query_binary_values('C1:WF? DAT2', datatype = 'B')
data = []
for d in raw_data:
    if d > 127:
        data.append(d - 256)
    else:
        data.append(d)
voltage = [d * (vdiv / 25) - voffset for d in data]

Finally, put the last three cells together to create a single function that captures a waveform and returns the following:
<ul>
    <li>tsample (float): sample time in seconds</li>
    <li>time (list): time array in seconds</li>
    <li>voltage (list): voltage array in V</li>
</ul>
Write a docstring for the function.

In [ ]:
def capture_data(inst):
    """
    Captures data from the most recent trigger 

    :param inst: pyvisa resource

    :return tsample: sample time in seconds 
    :return time: np.array, time array in seconds
    :return voltage: np.array, voltage array in V
    """
    fsample = inst.query('SARA?')
    fsample = float(fsample.split(' ')[1].replace('Sa/s\n', ''))
    tsample = 1 / fsample

    vdiv = get_yscale(inst)
    voffset = get_yoffset(inst)

    sleep(0.1)
    raw_data = inst.query_binary_values('C1:WF? DAT2', datatype = 'B')
    data = []
    for d in raw_data:
        if d > 127:
            data.append(d - 256)
        else:
            data.append(d)
    voltage = [d * (vdiv / 25) - voffset for d in data]

    time = np.arange(0, tsample * len(voltage), tsample)
    voltage = np.array(voltage)
    return tsample, time, voltage

### Note
If we were saving large amount of data, we may want to bypass creating the `time` array, and just save the sample time. For the small amounts of data we will use in this lab, it is easier to just create and save the time array with the voltage, so we don't have to keep track of `tsample`.

### Simple measurement
Using the functions you have defined above, we will perform a simple measurement. Suppose you have a signal of the frequency defined below. Initialize the instrument, then set the x scale to cover 20 periods of the signal (Note that the xscale is per division, and we have 14 divisions of data). Plot your output data, and create the log string.

In [ ]:
frequency = 1e3
tperiod = 1 / frequency 
total_time = tperiod * 10 
divisions = 14
xscale = total_time / divisions 

initialize(inst) 
set_xscale(inst, xscale)

In [ ]:
tsample, time, voltage = capture_data(inst)

In [ ]:
log = create_log(xscale, 0, 1, 0, 
                 1, 'AUTO')

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.set(ylabel = 'Voltage (V)', xlabel = 'Time (ms)')
ax.grid()
ax.plot(time * 1e3, voltage)

## FFT 
Perform an FFT on the data and plot the results on a log-log scale. It can help to omit the first point from the data on a log-log plot, because any DC offset will bias the y scale. Do the results match your expected signal?

In [ ]:
f, y = simple_fft(tsample, voltage)

In [ ]:
fig, ax = plt.subplots()
ax.set(ylabel = 'Voltage (V)', xlabel = 'Frequency (Hz)')
ax.set(yscale = 'log', xscale = 'log')
ax.grid()
ax.plot(f[1:], y[1:])

### Square wave FFT 
Try repeating the steps above with a square wave. Are the results what you expect?

In [ ]:
# set up instrument parameters
frequency = 1e3
tperiod = 1 / frequency 
total_time = tperiod * 10 
divisions = 14
xscale = total_time / divisions 
initialize(inst) 
set_xscale(inst, xscale)

# capture data
set_trigger_mode(inst, 'AUTO')
sleep(1)
tsample, time, voltage = capture_data(inst)

# create log file 
log = create_log(xscale, 0, 1, 0, 
                 1, 'AUTO')

# plot timestream
fig, ax = plt.subplots()
ax.set(ylabel = 'Voltage (V)', xlabel = 'Time (ms)')
ax.grid()
ax.plot(time * 1e3, voltage)
# plot FFT
f, y = simple_fft(tsample, voltage)
fig, ax = plt.subplots()
ax.set(ylabel = 'Voltage (V)', xlabel = 'Frequency (Hz)')
ax.set(yscale = 'log', xscale = 'log')
ax.grid()
ax.plot(f[1:], y[1:])

# Creating the .py file 
Finally, transfer all of your relevant functions into the siglent_sds1104xe.py file. This will allow us to use the code in the rest of the lab without copying/pasting into every notebook.

# Bonus
If you are familiar with Python classes, create a class to control the instrument that contains all of the functions you have written above. This is the standard method for writing instrument control code in Python.